In [4]:
import os
from smolagents import AzureOpenAIServerModel

model = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
    api_version=os.environ.get("OPENAI_API_VERSION", "")
)

In [5]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "medical_history").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [6]:
from medminer.task.history import history_task

for doc in docs:
    history_task.run(model, doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: history                                                                                              │
│ Prompt:                                                                                                         │
│                                                                                                                 │
│     Given a medical history of a patient, extract all given diagnoses and save all information as csv. The      │
│ diagnosis should be translated to english. The medical history is usually in the format of a sentence or a      │
│ paragraph. Every diagnosis should have a single row, if there are multiple diagnosis that can be extracted from │
│ a single piece of text, split them up. These are the steps you should follow to complete the task:              │
│     1. extract a part of the text that contains a diagnosis. The diagnosis can be in any language. This is the  │
│ diagnosis_reference column.                                                                                     │
│     2. translate the diagnoses to english if necessary and infer the diagnosis_translated column.               │
│     3. Extract the relevant diagnosis as a string and loose everything that is not relevant. This is the        │
│ diagnosis column.                                                                                               │
│     4. Extract the month and year of the medical history for that diagnosis. If not applicable, write an empty  │
│ string.                                                                                                         │
│                                                                                                                 │
│     Example 1:                                                                                                  │
│     Input: "schwere Herzinsuffizienz, 2. Grad, NYHA II, ED 07/2023"                                             │
│     Output: [{"patient_id": 1, "diagnosis_reference": "schwere chronische Herzinsuffizienz, 2. Grad, NYHA II",  │
│ "diagnosis_translated": "severe chronic heart failure, 2nd degree, NYHA II", "diagnosis": "chronic heart        │
│ failure", "month": "7", "year": "2023"}\]                                                                       │
│                                                                                                                 │
│     Example 2:                                                                                                  │
│     Input: "Myocardial Infarction"                                                                              │
│     Output: [{"patient_id": 2, "diagnosis_reference": "Myocardial Infarction", "diagnosis_translated":          │
│ "Myocardial Infarction", "diagnosis": "Myocardial Infarction", "month": "", "year": ""}\]                       │
│                                                                                                                 │
│     Example 3:                                                                                                  │
│     Input: "Lungenteilresektion rechts bei Lungenmetastasen eines Kolonkarzinoms, 2023-07-01"                   │
│     Output: [                                                                                                   │
│         {"patient_id": 3, "diagnosis_reference": "Lungenteilresektion rechts bei Lungenmetastasen eines         │
│ Kolonkarzinoms", "diagnosis_translated": "Lung resection due to lung metastases of colon cancer", "diagnosis":  │
│ "colon cancer", "month": "7", "year": "2023"},                                                                  │
│         {"patient_id": 3, "diagnosis_reference": "Lung

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'extract_diagnosis_data' with arguments: {'data': [{'patient_id': 3, 'diagnosis_reference':       │
│ 'koronare ZweigefÃ¤ÃŸerkrankung - NSTEMI 04/2017', 'diagnosis_translated': 'coronary two-vessel disease -       │
│ NSTEMI 04/2017', 'diagnosis': 'coronary two-vessel disease', 'month': '4', 'year': '2017'}, {'patient_id': 3,   │
│ 'diagnosis_reference': '90%ige Stenose des proximalen R. intraventicularis und 90-95%iger Stenose des           │
│ proximalen RCX', 'diagnosis_translated': '90% stenosis of the proximal R. intraventicularis and 90-95% stenosis │
│ of the proximal RCX', 'diagnosis': 'stenosis', 'month': '', 'year': ''}, {'patient_id': 3,                      │
│ 'diagnosis_reference': 'hochgradig reduzierte LV-Funktion bei Akinesie der anterolateralen Wand',               │
│ 'diagnosis_translated': 'severely reduced LV function with akinesia of the anterolateral wall', 'diagnosis':    │
│ 'reduced LV function', 'month': '', 'year': ''}, {'patient_id': 3, 'diagnosis_reference': 'Hypertonus',         │
│ 'diagnosis_translated': 'Hypertension', 'diagnosis': 'hypertension', 'month': '', 'year': ''}, {'patient_id':   │
│ 3, 'diagnosis_reference': 'hypertensive Herzerkrankung', 'diagnosis_translated': 'Hypertensive heart disease',  │
│ 'diagnosis': 'hypertensive heart disease', 'month': '', 'year': ''}, {'patient_id': 3, 'diagnosis_reference':   │
│ 'chron. Niereninsuffizienz', 'diagnosis_translated': 'Chronic renal insufficiency', 'diagnosis': 'renal         │
│ insufficiency', 'month': '', 'year': ''}]}                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'patient_id': 3, 'diagnosis_reference': 'koronare ZweigefÃ¤ÃŸerkrankung - NSTEMI 04/2017', 
'diagnosis_translated': 'coronary two-vessel disease - NSTEMI 04/2017', 'diagnosis': 'coronary two-vessel disease',
'month': '4', 'year': '2017'}, {'patient_id': 3, 'diagnosis_reference': '90%ige Stenose des proximalen R. 
intraventicularis und 90-95%iger Stenose des proximalen RCX', 'diagnosis_translated': '90% stenosis of the proximal
R. intraventicularis and 90-95% stenosis of the proximal RCX', 'diagnosis': 'stenosis', 'month': '', 'year': ''}, 
{'patient_id': 3, 'diagnosis_reference': 'hochgradig reduzierte LV-Funktion bei Akinesie der anterolateralen Wand',
'diagnosis_translated': 'severely reduced LV function with akinesia of the anterolateral wall', 'diagnosis': 
'reduced LV function', 'month': '', 'year': ''}, {'patient_id': 3, 'diagnosis_reference': 'Hypertonus', 
'diagnosis_translated': 'Hypertension', 'diagnosis': 'hypertension', 'month': '', 'year': ''}, {'patient_id': 3, 
'diagnosis_reference': 'hypertensive Herzerkrankung', 'diagnosis_translated': 'Hypertensive heart disease', 
'diagnosis': 'hypertensive heart disease', 'month': '', 'year': ''}, {'patient_id': 3, 'diagnosis_reference': 
'chron. Niereninsuffizienz', 'diagnosis_translated': 'Chronic renal insufficiency', 'diagnosis': 'renal 
insufficiency', 'month': '', 'year': ''}]

[Step 1: Duration 3.11 seconds| Input tokens: 2,173 | Output tokens: 357]

...

Error executing tool 'lookup_icd11' with arguments {"terms": ["coronary two-vessel disease", "stenosis", "reduced 
LV function", "hypertension", "hypertensive heart disease", "renal insufficiency"\]}: HTTPStatusError: Client error
'400 Bad Request' for url 
'https://id.who.int/icd/release/11/2022-02/mms/search?query=coronary+two-vessel+disease&useFlexisearch=true'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400
Please try again or use another tool

[Step 2: Duration 4.21 seconds| Input tokens: 5,247 | Output tokens: 776]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 3: Duration 25.67 seconds| Input tokens: 8,321 | Output tokens: 1,195]

KeyboardInterrupt: 